## OnlineandSD_step1
### Author: Olivia Sablan
The following code is used to read in files from the PurpleAir Data Download Tool and files removed from the PurpleAir SD card and compile the data. 

Last updated: May 19, 2025

In [33]:
import pandas as pd
import numpy as np
import glob
import os
import warnings
import pathlib
from datetime import datetime
pd.options.mode.copy_on_write = True
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
# This reads the google spreadsheet with all the field sensor info as a csv
google = pd.read_csv('https://docs.google.com/spreadsheets/')
# Drop the sensors that haven't been registered and have no ID to pull data from and make it an integer
google = google.dropna(subset = ['Sensor ID to Pull'])
google['Sensor ID to Pull'] = google['Sensor ID to Pull'].astype(int)

# To loop through the sensor IDs, I make lists of the ID used to pull them as an integer and a string
senslist=np.array(google['Sensor ID to Pull'][:],dtype='int')
strsenslist= [str(i) for i in senslist]

# Because some sensors were put in multiple locations (but have the same ID to pull) I make another list 
# for the 'Sensor ID for User' to save the data separately when a sensor was used twice
senslist2=np.array(google['Sensor ID for User'][:],dtype='int')
strsenslist2= [str(i) for i in senslist2]

In [21]:
# this is the directory where I put the data I downloaded from the PurpleAir Data Download Tool
# note: this is only online data for the sensors that were on WiFi and could report their data to PurpleAir storage
dir_list = os.listdir('../data/datadownloadtool/')
# We don't need "DS.store" to be in the list because it doesn't have data in it
dir_list = [file for file in dir_list if file != '.DS_Store']

In [22]:
# I pulled data every month, and the data download tool stores the .csv by the dates of the data 
# I store the end of the names of the csv's here so I can loop through each csv later
endofstring = []
for i in range(len(dir_list)):
    endofstring.append(os.listdir('../data/datadownloadtool/' + dir_list[i])[1][6:])

In [23]:
# same as above, but I want to store the exact date I pulled the data, which is only characters 19:29
dates_pulled = []
for i in range(len(dir_list)):
    dates_pulled.append(dir_list[i][19:29])

In [24]:
# Need all the lists to iterate through in the right order
dates_pulled.sort(key=lambda date: datetime.strptime(date, '%m-%d-%Y')) 
endofstring.sort()
dir_list.sort(key=lambda x: datetime.strptime(x.split()[-1], '%m-%d-%Y'))

### Compiling data from the PurpleAir Data Download Tool:
Note: this is only data that was stored on the PurpleAir servers from sensors that were on WiFi. This may not include all of the data collected by the sensors

In [25]:
online_df = pd.DataFrame()

# start looping through each data file that contains all IDs
for ii in range(len(dir_list)):
    for i in range(len(strsenslist)):
        file = pathlib.Path('../data/datadownloadtool/' + dir_list[ii] + '/' + strsenslist[i] + endofstring[ii])
        if file.exists():  # Only continue if the sensor file exists
            df = pd.read_csv(file)
            if len(df) > 0:  # Only continue if there's data in the sensor file
                df.rename(columns={'time_stamp': 'created_at'}, inplace=True)  # Rename to match previous data
                df['created_at'] = pd.to_datetime(df['created_at'], format="%Y-%m-%dT%H:%M:%Sz")  # Convert to datetime
                sensor_instances = google[google['Sensor ID to Pull'] == senslist[i]]
                if len(sensor_instances) > 1:
                    start1 = google['Start'][google['Sensor ID to Pull'] == senslist[i]].iloc[0]
                    end1 = google['Stop'][google['Sensor ID to Pull'] == senslist[i]].iloc[0]
                    start2 = google['Start'][google['Sensor ID to Pull'] == senslist[i]].iloc[1]
                    # First time sensor was used
                    sensor1 = df[(df['created_at'].dt.strftime('%Y/%m/%d') >= start1) & (df['created_at'].dt.strftime('%Y/%m/%d') <= end1)]
                    ten1 = sensor1.groupby(pd.Grouper(key="created_at", freq="10min")).mean(numeric_only=True).reset_index()
                    ten1['ID'] = strsenslist[i]
                    # Second time sensor was used
                    sensor2 = df[(df['created_at'].dt.strftime('%Y/%m/%d') >= start2) ]
                    ten2 = sensor2.groupby(pd.Grouper(key="created_at", freq="10min")).mean(numeric_only=True).reset_index()
                    ten2['ID'] = strsenslist[i] + '2'
                    online_df = pd.concat([online_df, ten1])
                    online_df = pd.concat([online_df, ten2])
                else:
                    start1 = google['Start'][google['Sensor ID to Pull'] == senslist[i]].iloc[0]
                    end1 = google['Stop'][google['Sensor ID to Pull'] == senslist[i]].iloc[0]

                    if pd.isna(end1):
                        df = df[df['created_at'].dt.strftime('%Y/%m/%d') >= start1]
                        ten3 = df.groupby(pd.Grouper(key="created_at", freq="10min")).mean(numeric_only = True).reset_index()
                        ten3['ID'] = strsenslist[i]
                        online_df = pd.concat([online_df, ten3])
                    else:
                        df = df[(df['created_at'].dt.strftime('%Y/%m/%d') >= start1) & (df['created_at'].dt.strftime('%Y/%m/%d') <= end1)]
                        ten4 = df.groupby(pd.Grouper(key="created_at", freq="10min")).mean(numeric_only = True).reset_index()
                        ten4['ID'] = strsenslist[i]
                        online_df = pd.concat([online_df, ten4])

### Compiling data from the PurpleAir SD cards:

In [18]:
SD_df = pd.DataFrame()
for i in range(len(senslist)):
    inputfiles = glob.glob('../data/SD_card/'+ strsenslist[i] +'/*.csv') # Files from the SD cards are put in one directory for each sensor
    if (len(inputfiles) > 0): # only continue if there is a datafile for the particular sensor
        df_from_each_file = (pd.read_csv(f) for f in inputfiles)
        df = pd.concat(df_from_each_file,sort=False)
        df = df[['UTCDateTime', 'current_humidity', 'current_temp_f', 'pm2_5_cf_1', 'pm2_5_cf_1_b']] #keep only necessary coljumns
        df = df.rename(columns = {'UTCDateTime': 'created_at', 'current_humidity' : 'humidity', #rename to match previous data
                                  'current_temp_f':'temperature', 'pm2_5_cf_1' : 'pm2.5_cf_1_a', 'pm2_5_cf_1_b':'pm2.5_cf_1_b'})
        df['created_at'] =  pd.to_datetime(df['created_at'], format="%Y/%m/%dT%H:%M:%Sz")
        start_date = google['Start'][google['Sensor ID to Pull'] == senslist[i]].iloc[0] #store the start date of measurements
        df = df[df['created_at'].dt.strftime('%Y/%m/%d') >= start_date] # only keep data after "start_date" of measurements
        df['humidity'] = df['humidity'].astype('float')
        df['temperature'] = df['temperature'].astype('float')
        ten = df.groupby(pd.Grouper(key="created_at", freq="10min")).mean(numeric_only = True).reset_index() #take 10-min averages of the data
        SD_df = pd.concat([SD_df, ten4])

### Combining the online and SD card data:

In [38]:
combined_SD_online = pd.DataFrame()
print('If the following values are not zero, something is wrong.')
for i in range(len(senslist)):
    online_onesensor = online_df[online_df['ID'] == strsenslist2[i]]
    SD_onesensor = SD_df[SD_df['ID'] == strsenslist2[i]]
    if (len(online_onesensor) > 0) & (len(SD_onesensor) > 0):
            combined_df = pd.concat([SD_onesensor[['created_at', 'humidity', 'temperature', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b']], 
                                     online_onesensor[['created_at', 'humidity', 'temperature', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b']]]).drop_duplicates(subset='created_at', keep= 'first')
            combined_df['ID'] = senslist2[i]
            combined_SD_online = pd.concat([combined_SD_online, combined_df])
    elif (len(online_onesensor) > 0) & (len(SD_onesensor) == 0):
            online_onesensor['ID'] = senslist2[i]
            combined_SD_online = pd.concat([combined_SD_online, online_onesensor])
    elif (len(online_onesensor) == 0) & (len(SD_onesensor) > 0):
            SD_onesensor['ID'] = senslist2[i]
            combined_SD_online = pd.concat([combined_SD_online, combined_df])
    else: # These should only ever be 0! otherwise it should be saved in the other categories
        print('online pull: ', len(online_onesensor), 
              'SD card: ', len(SD_onesensor))
combined_SD_online.to_csv('../data/raw_combinedSDonline.csv')

If the following values are not zero, something is wrong
online pull:  0 SD card:  0
online pull:  0 SD card:  0
online pull:  0 SD card:  0
